In [28]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as anim
import numpy as np
plt.rcParams['animation.ffmpeg_path'] = '/Users/chris/Downloads/ffmpeg'
c1 = mpl.colormaps['viridis']

cmds = open('./data.txt', 'r').read()

sw = 7

def enlarge_stage(stage, space_needed, max_rock):
    clear_rows = len(stage) - max_rock - 1
    if clear_rows > space_needed + 3:
        return
    stage.extend([[0] * sw for i in range(space_needed + 3 - clear_rows)])

def shrink_stage(stage, new_min):
    return stage[new_min:]

def find_max(stage):
    for i in range(len(stage) - 1, -1, -1):
        if any(stage[i]):
            return i

def find_full(stage):
    for i in range(len(stage) - 1, -1, -1):
        if all([a or b for a, b in zip(stage[i], stage[i - 1])]):
            return i - 1

class Wide:
    @staticmethod
    def height():
        return 1
    @staticmethod
    def l(stage, x, y):
        if x == 0:
            return True
        return stage[y][x - 1] > 0
    @staticmethod
    def r(stage, x, y):
        if x == sw - 4:
            return True
        return stage[y][x + 4] > 0
    @staticmethod
    def d(stage, x, y):
        return any(map(lambda i: stage[y - 1][x + i] > 0, range(4)))
    @staticmethod
    def insert_at(stage, x, y):
        for i in range(4):
            stage[y][x + i] = 1

class Tall:
    @staticmethod
    def height():
        return 4
    @staticmethod
    def l(stage, x, y):
        if x == 0:
            return True
        return any(map(lambda i: stage[y + i][x - 1] > 0, range(4)))
    @staticmethod
    def r(stage, x, y):
        if x == sw - 1:
            return True
        return any(map(lambda i: stage[y + i][x + 1] > 0, range(4)))
    @staticmethod
    def d(stage, x, y):
        return stage[y - 1][x] > 0
    @staticmethod
    def insert_at(stage, x, y):
        for i in range(4):
            stage[y + i][x] = 4

class Square:
    @staticmethod
    def height():
        return 2
    @staticmethod
    def l(stage, x, y):
        if x == 0:
            return True
        return any(map(lambda i: stage[y + i][x - 1] > 0, range(2)))
    @staticmethod
    def r(stage, x, y):
        if x == sw - 2:
            return True
        return any(map(lambda i: stage[y + i][x + 2] > 0, range(2)))
    @staticmethod
    def d(stage, x, y):
        return any(map(lambda i: stage[y - 1][x + i] > 0, range(2)))
    @staticmethod
    def insert_at(stage, x, y):
        for j in range(2):
            for i in range(2):
                stage[y + i][x + j] = 5

class Plus:
    @staticmethod
    def height():
        return 3
    @staticmethod
    def l(stage, x, y):
        if x == 0:
            return True
        tests = [(x, y), (x - 1, y + 1), (x, y + 2)]
        return any(map(lambda p: stage[p[1]][p[0]] > 0, tests))
    @staticmethod
    def r(stage, x, y):
        if x == sw - 3:
            return True
        tests = [(x + 2, y), (x + 3, y + 1), (x + 2, y + 2)]
        return any(map(lambda p: stage[p[1]][p[0]] > 0, tests))
    @staticmethod
    def d(stage, x, y):
        tests = [(x, y), (x + 1, y - 1), (x + 2, y)]
        return any(map(lambda p: stage[p[1]][p[0]] > 0, tests))
    @staticmethod
    def insert_at(stage, x, y):
        stage[y][x + 1] = 2
        stage[y + 2][x + 1] = 2
        for i in range(3):
            stage[y + 1][x + i] = 2

class El:
    @staticmethod
    def height():
        return 3
    @staticmethod
    def l(stage, x, y):
        if x == 0:
            return True
        tests = [(x - 1, y), (x + 1, y + 1), (x + 1, y + 2)]
        return any(map(lambda p: stage[p[1]][p[0]] > 0, tests))
    @staticmethod
    def r(stage, x, y):
        if x == sw - 3:
            return True
        tests = [(x + 3, y), (x + 3, y + 1), (x + 3, y + 2)]
        return any(map(lambda p: stage[p[1]][p[0]] > 0, tests))
    @staticmethod
    def d(stage, x, y):
        tests = [(x, y - 1), (x + 1, y - 1), (x + 2, y - 1)]
        return any(map(lambda p: stage[p[1]][p[0]] > 0, tests))
    @staticmethod
    def insert_at(stage, x, y):
        stage[y + 1][x + 2] = 3
        stage[y + 2][x + 2] = 3
        for i in range(3):
            stage[y][x + i] = 3

shapes = [Wide, Plus, El, Tall, Square]

In [15]:
test_cmds = '>>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>'

def run_tetris(stage, cmds, stop_at):
    fallen_rocks = 0

    shape_index = 0
    cur_shape = shapes[shape_index]

    c_i = -1

    def get_start():
        max = find_max(stage)
        needed = 3 + cur_shape.height()
        enlarge_stage(stage, needed, max)
        return (2, max + 4)

    x, y = get_start()

    while fallen_rocks < stop_at:
        c_i += 1
        if c_i >= len(cmds):
            c_i = 0
        cm = cmds[c_i]
        if cm == '<':
            x = x - 1 if not cur_shape.l(stage, x, y) else x
        elif cm == '>':
            x = x + 1 if not cur_shape.r(stage, x, y) else x
        
        down = cur_shape.d(stage, x, y)
        if down:
            fallen_rocks += 1
            cur_shape.insert_at(stage, x, y)
            shape_index += 1
            if shape_index >= len(shapes):
                shape_index = 0
            cur_shape = shapes[shape_index]
            x, y = get_start()
        else:
            y = y - 1
    
    return stage

stage = [[6] * sw]
run_tetris(stage, cmds, 2022)
find_max(stage)

3153

In [77]:
p2_stop = 1000000000000

def run_tetris2(stage_in, cmds, stop_at):
    stage = [row for row in stage_in]
    fallen_rocks = 0

    shape_index = 0
    cur_shape = shapes[shape_index]
    stage_min = 0

    test_set = set()
    streak = []

    c_i = -1

    def get_start():
        max = find_max(stage)
        needed = 3 + cur_shape.height()
        enlarge_stage(stage, needed, max)
        return (2, max + 4)
    
    # def try_shrink():
    #     new_min = find_full(stage)
    #     if new_min:
    #         # print(f'Found new min {stage_min} {new_min}')
    #         return shrink_stage(stage, new_min), stage_min + new_min, y - new_min
    #     else:
    #         # print(f'No new min found')
    #         return stage, stage_min, y

    x, y = get_start()
    cur_moves = ''

    while fallen_rocks < stop_at:
        c_i += 1
        if c_i >= len(cmds):
            c_i = 0
        cm = cmds[c_i]
        if cm == '<':
            if not cur_shape.l(stage, x, y):
                x = x - 1
                cur_moves += 'l'
        elif cm == '>':
            if not cur_shape.r(stage, x, y):
                x = x + 1
                cur_moves += 'd'
        
        if cur_shape.d(stage, x, y):
            fallen_rocks += 1
            # if not fallen_rocks % 1000:
            #     stage, stage_min, y = try_shrink()
            cur_shape.insert_at(stage, x, y)
            shape_index += 1
            if shape_index >= len(shapes):
                shape_index = 0
            cur_shape = shapes[shape_index]

            match = (shape_index, c_i + 1, cur_moves)
            if match not in test_set:
                test_set.add(match)
                streak = []
            else:
                mi = find_max(stage)
                item = (match, fallen_rocks, mi)
                if len(streak) > 1 and streak[0][0] == match:
                    s1, s2, s3 = (streak[0], item, streak[952])
                    print(f'loop detected {s1} {s2} {s3} {s2[1] - s1[1]} {s2[2] - s1[2]} {s3[1] - s1[1]} {s3[2] - s1[2]}')
                    streak = []
                streak.append(item)
            
            cur_moves = ''
            
            x, y = get_start()
        else:
            y = y - 1
            cur_moves += 'd'
    
    return stage, stage_min

stage2 = [[6] * sw]
new_stage, new_min = run_tetris2(stage2, cmds, 100000)

print(f'{len(new_stage)} {new_min} {len(new_stage) + new_min}')

loop detected ((3, 3642, 'ldlddd'), 2338, 3658) ((3, 3642, 'ldlddd'), 4043, 6307) ((0, 9252, 'ddddldl'), 3290, 5117) 1705 2649 952 1459
loop detected ((3, 3642, 'ldlddd'), 4043, 6307) ((3, 3642, 'ldlddd'), 5748, 8956) ((0, 9252, 'ddddldl'), 4995, 7766) 1705 2649 952 1459
loop detected ((3, 3642, 'ldlddd'), 5748, 8956) ((3, 3642, 'ldlddd'), 7453, 11605) ((0, 9252, 'ddddldl'), 6700, 10415) 1705 2649 952 1459
loop detected ((3, 3642, 'ldlddd'), 7453, 11605) ((3, 3642, 'ldlddd'), 9158, 14254) ((0, 9252, 'ddddldl'), 8405, 13064) 1705 2649 952 1459
loop detected ((3, 3642, 'ldlddd'), 9158, 14254) ((3, 3642, 'ldlddd'), 10863, 16903) ((0, 9252, 'ddddldl'), 10110, 15713) 1705 2649 952 1459
loop detected ((3, 3642, 'ldlddd'), 10863, 16903) ((3, 3642, 'ldlddd'), 12568, 19552) ((0, 9252, 'ddddldl'), 11815, 18362) 1705 2649 952 1459
loop detected ((3, 3642, 'ldlddd'), 12568, 19552) ((3, 3642, 'ldlddd'), 14273, 22201) ((0, 9252, 'ddddldl'), 13520, 21011) 1705 2649 952 1459
loop detected ((3, 3642, '

In [78]:
rocks = 99523
height = 154651

loops = (p2_stop - rocks) // 1705
mod = p2_stop - rocks - (loops * 1705)
print(mod)
(loops * 2649) + height + 1459

952


1553665689155

In [10]:
def create_image(stage):
    fig, ax = plt.subplots(1, 1, figsize=(5, 5 * (len(stage) / 70)))
    ax.pcolormesh(stage, cmap=c1, rasterized=True, vmin=0, vmax=6)